In [2]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv('./Data/mitbih_train.csv')
df2 =pd.read_csv('./Data/mitbih_test.csv')

In [3]:

X = df.iloc[:, :-1]  # All columns except the last
Y = df.iloc[:, -1]   # Only the last column

# Check the shapes
print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")
X

Shape of X: (87553, 187)
Shape of Y: (87553,)


,9.779411554336547852e-01,9.264705777168273926e-01,6.813725233078002930e-01,2.450980395078659058e-01,1.544117629528045654e-01,1.911764740943908691e-01,1.519607901573181152e-01,8.578431606292724609e-02,5.882352963089942932e-02,4.901960864663124084e-02,...,0.000000000000000000e+00.78,0.000000000000000000e+00.79,0.000000000000000000e+00.80,0.000000000000000000e+00.81,0.000000000000000000e+00.82,0.000000000000000000e+00.83,0.000000000000000000e+00.84,0.000000000000000000e+00.85,0.000000000000000000e+00.86,0.000000000000000000e+00.87
0,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.927461,1.000000,0.626943,0.193437,0.094991,0.072539,0.043178,0.053541,0.093264,0.189983,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87548,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87549,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87550,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87551,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
Xtest,Ytest = df2.iloc[:, :-1],df2.iloc[:, -1]

In [5]:
import torch
import torch.nn as nn
class myModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(myModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 16),  # First dense layer
            nn.ReLU(),                  # Activation function
            nn.Dropout(0.2),            # Dropout for regularization
            nn.Linear(16, 5),         # Second dense layer
            # nn.ReLU(),                  # Activation function
            # nn.Dropout(0.2),            # Dropout for regularization
            # nn.Linear(64, num_classes)  # Output layer for classification
        )

    def forward(self, x):
        return self.model(x)



In [6]:

input_size = 187  # Number of features in X
num_classes = 5
model = myModel(input_size=input_size, num_classes=num_classes)

In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)  # Convert X to tensor
Y_tensor = torch.tensor(Y.values, dtype=torch.long)     # Convert Y to tensor

# Create DataLoader for batching
batch_size = 64
dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()  # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU (if CUDA is available)
model.to(device)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        # Move data to GPU (if CUDA is available)
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Print epoch statistics
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")


Epoch [1/20], Loss: 0.9317
Epoch [2/20], Loss: 0.6516
Epoch [3/20], Loss: 0.5323
Epoch [4/20], Loss: 0.4759
Epoch [5/20], Loss: 0.4499
Epoch [6/20], Loss: 0.4306
Epoch [7/20], Loss: 0.4092
Epoch [8/20], Loss: 0.3948
Epoch [9/20], Loss: 0.3837
Epoch [10/20], Loss: 0.3749
Epoch [11/20], Loss: 0.3672
Epoch [12/20], Loss: 0.3621
Epoch [13/20], Loss: 0.3578
Epoch [14/20], Loss: 0.3511
Epoch [15/20], Loss: 0.3471
Epoch [16/20], Loss: 0.3427
Epoch [17/20], Loss: 0.3382
Epoch [18/20], Loss: 0.3322
Epoch [19/20], Loss: 0.3259
Epoch [20/20], Loss: 0.3212
Training complete!


In [10]:
# Convert test data to tensors
Xtest_tensor = torch.tensor(Xtest.values, dtype=torch.float32)
Ytest_tensor = torch.tensor(Ytest.values, dtype=torch.long)
print(f"Shape of X: {Xtest_tensor.shape}")
print(f"Shape of Y: {Ytest_tensor.shape}")

Shape of X: torch.Size([21891, 187])
Shape of Y: torch.Size([21891])


In [11]:

model.to(device)

Xtest_tensor = Xtest_tensor.to(device)
Ytest_tensor = Ytest_tensor.to(device)

model.eval()  
with torch.no_grad():  

    outputs = model(Xtest_tensor) 
    _, predicted = torch.max(outputs, 1)  

    correct = (predicted == Ytest_tensor).sum().item()
    total = Ytest_tensor.size(0)
    accuracy = correct / total * 100

print(f"Accuracy on test set: {accuracy:.2f}%")


Accuracy on test set: 91.96%


In [12]:
torch.save(model.state_dict(),"myModel.pth")
# model.cpu()  # Move model to CPU
# dummy_input = torch.randn(1, 187)  # Dummy input on CPU
# traced_model = torch.jit.trace(model, dummy_input)
# traced_model.save("traced_model_cpu.pt")


In [13]:

Xtest_tensor_fp16 = Xtest_tensor.half()
model.half()  

model.eval()
with torch.no_grad():  

    outputs_fp16 = model(Xtest_tensor_fp16) 
    _, predicted_fp16 = torch.max(outputs_fp16, 1)  

    correct_fp16 = (predicted_fp16 == Ytest_tensor).sum().item()
    total_fp16 = Ytest_tensor.size(0)
    accuracy_fp16 = correct_fp16 / total_fp16 * 100

print(f"Accuracy on test set with float16: {accuracy_fp16:.2f}%")


Accuracy on test set with float16: 91.96%


In [15]:
torch.save(model.state_dict(), "myModelHalf.pth")


In [17]:
Xtest_tensor_bf16 = Xtest_tensor.to(dtype=torch.bfloat16)
model.to(dtype=torch.bfloat16)

model.eval()
with torch.no_grad():
    outputs_bf16 = model(Xtest_tensor_bf16)
    _, predicted_bf16 = torch.max(outputs_bf16, 1)
    correct_bf16 = (predicted_bf16 == Ytest_tensor).sum().item()
    total_bf16 = Ytest_tensor.size(0)
    accuracy_bf16 = correct_bf16 / total_bf16 * 100

print(f"Accuracy on test set with bfloat16: {accuracy_bf16:.2f}%")


Accuracy on test set with bfloat16: 91.97%
